In [ ]:
'''
Moshe's idea:
1. Start with a large Boolean formula f (say, industrial SAT benchmark).
2. Generate many samples satisfying f or \neg f.
(Using, for example, [https://www.nature.com/articles/s42256-018-0002-3](https://www.nature.com/articles/s42256-018-0002-3))
3. Train a DNN on this labeled sample.
4. Check whether it is a good apprcximation of f.
----------
To use this notebook, please install pysat: https://pysathq.github.io/installation.html
Perhaps the simplest way is with the following command: pip install python-sat[pblib,aiger]
(pysat might not work on Windows)
'''


# This is just a warm-up cell, to see how pysat works and to generate random unsat assignments

# some examples of pysat usage at: https://github.com/pysathq/pysat/blob/master/examples/usage.py
from pysat.solvers import Glucose3
from pysat.formula import CNF
import random
f = CNF(from_clauses=[[-1, 2], [3]])

g = Glucose3(bootstrap_with=f)

positives = list(g.enum_models())
print('f: ', f.clauses)
print('positives: ', positives)
g.delete()

negatives = set()
while len(negatives) < len(positives) and len(negatives) + len(positives) < 6:
    # generates a random assignment and adds it to the list of negatives if
    # it does NOT satisfy f
    candidate = [x * random.choice([-1, 1]) for x in range(1, f.nv+1)]
    # problematic one is: candidate:  [-1, -2, 3]
    print('candidate: ', candidate)
    
    # avoids generating duplicate instances
    if tuple(candidate) in negatives:
        #print(f'duplicate {candidate} detected')
        continue

    for clause in f.clauses:
        # print('clause:', clause)
        # if multiplication is positive, then literal evaluates to true
        c_true = False
        for l in clause: # l stands for 'literal'
            c_true = c_true or candidate[abs(l)-1] * l > 0
            print(f'l={l}, c={candidate[abs(l)-1]}, l*c<0? {candidate[abs(l)-1] * l < 0}, cTrue? {c_true}')
            
        # if variable and literal disagree on sign, result is False; if all are false in the clause
        # we found a unsatisfying assignment
        if all([candidate[abs(l)-1] * l < 0 for l in clause]) == False: 
            print(f'{candidate} evaluated to false, adding to negatives')
            negatives.add(tuple(candidate))
            break
print('negatives:', negatives)
        #satisfied = all([any(x) for ])


In [ ]:
# to use this cell, extract bw_large.d.cnf from instances/blocksworld.tar.gz
# blocksworld is from satlib at https://www.cs.ubc.ca/~hoos/SATLIB/Benchmarks/SAT/PLANNING/BlocksWorld/blocksworld.tar.gz
# satlib website: https://www.cs.ubc.ca/~hoos/SATLIB/benchm.html

import random
import pandas as pd
from pysat.solvers import Glucose3, Solver, NoSuchSolverError
from pysat.formula import CNF
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score


def generate_dataset(cnf_file, solver_name='Glucose3', max_samples=10):
    """
    Generates a dataset from the boolean formula in the informed CNF file.
    Particularly, it enumerates all satisfying samples with a SAT solver.
    Then it generates the same number of unsat samples.
    Unsat samples are generated by flipping one bit of a sat sample.

    All samples are shuffled and assembled into two dataframes.
    The first dataframe contains the binary (0/1) inputs.
    The second dataframe contains the corresponding binary label (0/1) for unsat/sat,
    respectively.

    :param cnf_file: path to the file in CNF (Dimacs) format (see https://people.sc.fsu.edu/~jburkardt/data/cnf/cnf.html)
    :param solver_name: name of the solver to generate the positive instances
    :param max_samples: maximum number of samples, half of them will be positive, half negative
    :return: tuple with 2 dataframes: inputs and labels
    """
    
    print(f'Reading boolean formula from {cnf_file}.')
    formula = CNF(from_file=cnf_file)

    # num_vars = formula.nv
    unsat_list = []

    print(f'Finding satisfiable assignments with {solver_name}')
    try:
        with Solver(name=solver_name, bootstrap_with=formula) as solver:

            # for each positive (sat) instance, flip a literal to generate a negative (unsat) instance

            # generates the satisfying instances by querying the solver
            sat_list = []
            for i, solution in enumerate(solver.enum_models()):
                sat_list.append(solution)
                if i+1 >= max_samples / 2:  # adds 1 to index as it starts at zero
                    print(f"Limit number of {max_samples/2} positive samples reached.")
                    break

            print(f'Found {len(sat_list)} sat instances.')
            if len(sat_list) == 0:
                print('WARNING: No sat instance found, returning empty dataset.')
                return [], []

            print('Generating the same number of unsat instances.')

            # transforming each sat instance into tuple eases the generation of negative instances
            sat_set = set([tuple(s) for s in sat_list])  # set is much quicker to verify an existing instance

            for assignment in sat_list:
                for i, literal in enumerate(assignment):
                    tentative_unsat = list(assignment)
                    tentative_unsat[i] = -tentative_unsat[i]  # negating one literal
                    if tuple(tentative_unsat) not in sat_set:
                        unsat_list.append(tentative_unsat)
                        break  # goes on to next assignment
                    # print(f'negated {i}-th')
    except NoSuchSolverError as e:
        print(f'ERROR: no solver named "{solver_name}" was found. '
              f'Please use one of the names in '
              f'https://pysathq.github.io/docs/html/api/solvers.html#pysat.solvers.SolverNames'
              f', such as Glucose3, for example. Exiting.')
        sys.exit(0)

    # sat_list.append(sat_list[0]) ## uncomment to test duplicates
    
    # appends the labels (1 to sat samples, 0 to unsat samples)
    for sat in sat_list:
        sat.append(1)
    for unsat in unsat_list:
        unsat.append(0)

    print(f'Preparing dataset.')
    # concats and shuffles the two lists
    all_data = sat_list + unsat_list
    # random.seed(2) # uncomment to debug (otherwise each shuffle will give a different array)
    random.shuffle(all_data)

    # column names = [x1, x2, ..., xn, f] (each x_i is a variable and f is the label)
    input_names = [f'x{i}' for i in range(1, len(all_data[0]))]
    df = pd.DataFrame(all_data, columns=input_names + ['f'])
    if any(df.duplicated(input_names)):
        print('ERROR: there are duplicate entries in the dataset. Returning empty.')
        return [], []

    # replaces negatives by 0 and positives by 1
    df.mask(df < 0, 0, inplace=True)
    df.mask(df > 0, 1, inplace=True)

    # breaks into input features & label
    data_x = df.drop('f', axis=1)
    data_y = df['f']

    return data_x, data_y

def run_model(model, data_x, data_y, splitter=StratifiedKFold(n_splits=5)):
    '''
    Runs a machine learning model in the dataset
    '''
    
    # prints the header
    print('#instances\tprec\tacc')

    # trains the model for each split (fold)
    for train_index, test_index in splitter.split(data_x, data_y):
        # splits the data frames in test and train
        train_x, train_y = data_x.iloc[train_index], data_y.iloc[train_index]
        test_x, test_y = data_x.iloc[test_index], data_y.iloc[test_index]
        
        model.fit(train_x, train_y)

        # obtains the predictions on the test set
        predictions = model.predict(test_x)

        # calculates and reports some metrics
        acc = accuracy_score(predictions, test_y)
        prec = precision_score(predictions, test_y, average='macro')
        print('{}\t\t{:.3f}\t{:.3f}'.format(len(test_y), prec, acc))
 


data_x, data_y = generate_dataset('instances/bw_large.d.cnf')

print('Decision tree')
learner = DecisionTreeClassifier()
run_model(learner, data_x, data_y ) #default is stratified 5-fold cross validati
#run_model(learner, data_x, data_y, StratifiedShuffleSplit(n_splits=1, test_size=0.10)) # test with holdout of 10% for test

In [ ]:
# generate dataset using unigen2
import re
import os

def generate_dataset(cnf_file, max_samples=1000):
    """
    Uses Unigen2 (https://bitbucket.org/kuldeepmeel/unigen) to generate a dataset
    :param cnf_file:
    :param max_samples:
    :return:
    """
    import subprocess
    
    tmp_dir = '/tmp/unigen'

    # makes sure that unigen working dir exists
    os.makedirs(tmp_dir, exist_ok=True)

    '''
    unigen is called with: python UniGen2.py [options] cnf_file output
    We'll call python from the command line as it is not straightforward to 
    import unigen and instantiate its parameters directly
    '''

    # saves the current dir, the input name (filename w/o extension) and
    # absolute path to the input
    current_dir = os.getcwd()
    cnf_name = os.path.splitext(os.path.basename(cnf_file))[0]
    cnf_abspath = os.path.abspath(cnf_file)

    # enters unigen dir and generates samples for f and ~f
    #os.chdir('unigen')
    # generates positive samples (calls unigen on the formula from cnf_file)
    '''
    ret_code = subprocess.call(
        f'python UniGen2.py -samples={max_samples//2} -runIndex=0 '
        f'{cnf_abspath} {self.tmp_dir}'.split(' ')  # split because call accepts an array
    )
    if ret_code != 0:
        print("WARNING! there has been some error with unigen's execution. "
              "Please check the output above.")
    '''
    # the file with the positive samples is self.tmp_dir/cnf_file_0.txt
    with open(os.path.join(tmp_dir, f'{cnf_name}_0.txt')) as sample_file:
        for line in sample_file.readlines():
            # format of each line: v1 2 ... N 0:M, where 1 2 ... N are the variables (asserted or negated)
            #print(line)
            #match = re.match('v(\d*)', line.strip())
            str_assignments = line.strip()[1:].split(' ')[:-1]
            

    # comes back to the original dir
    os.chdir(current_dir)
    
generate_dataset('instances/bw_large.d.cnf')